# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

## Store Part I results into DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
csv_file = os.path.join('..', 'WeatherPy', 'Output', 'city_weather.csv')
city_df = pd.read_csv(csv_file)
city_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,964432,Port Alfred,ZA,-33.5906,26.8910,60.10,75,98,6.87
1,1326105,Falam,MM,22.9167,93.6833,75.16,76,91,1.14
2,5563839,Fortuna,US,40.5982,-124.1573,55.67,73,90,1.01
3,336350,Goba,ET,7.0167,39.9833,62.67,53,29,4.97
4,3430863,Mar del Plata,AR,-38.0023,-57.5575,54.07,65,90,3.00
...,...,...,...,...,...,...,...,...,...
568,3393692,Itarema,BR,-2.9248,-39.9167,73.53,85,8,9.64
569,3517970,Pochutla,MX,15.7432,-96.4661,74.82,86,100,5.06
570,641489,Pirkkala,FI,61.4613,23.6323,61.61,86,90,5.01
571,2057087,Kupang,ID,-10.1667,123.5833,88.38,55,40,12.66


## Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

# Add Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

In [4]:
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
24,1513491,Yangi Marg`ilon,UZ,40.4272,71.7189,86.72,16,0,6.91
59,1529660,Aksu,CN,41.1231,80.2644,85.69,19,0,4.29
74,510347,Pestravka,RU,52.4050,49.9583,85.23,54,0,5.46
88,2036283,Kuandian,CN,40.7286,124.7847,76.73,57,3,4.65
103,1280037,Shache,CN,38.4167,77.2406,89.51,9,0,5.28
123,304797,Mardin,TR,37.3131,40.7436,78.06,39,0,4.61
130,233019,Kaberamaido,UG,1.7389,33.1594,75.88,63,0,1.86
177,4669635,Abilene,US,32.4487,-99.7331,81.66,63,1,1.99
232,1253628,Ginda,IN,30.6945,78.4932,83.17,32,3,6.82
245,542464,Krasnoarmeysk,RU,51.0239,45.6969,82.54,57,3,8.16


## Hotel Map

In [5]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
24,1513491,Yangi Marg`ilon,UZ,40.4272,71.7189,86.72,16,0,6.91,
59,1529660,Aksu,CN,41.1231,80.2644,85.69,19,0,4.29,
74,510347,Pestravka,RU,52.4050,49.9583,85.23,54,0,5.46,
88,2036283,Kuandian,CN,40.7286,124.7847,76.73,57,3,4.65,
103,1280037,Shache,CN,38.4167,77.2406,89.51,9,0,5.28,
123,304797,Mardin,TR,37.3131,40.7436,78.06,39,0,4.61,
130,233019,Kaberamaido,UG,1.7389,33.1594,75.88,63,0,1.86,
177,4669635,Abilene,US,32.4487,-99.7331,81.66,63,1,1.99,
232,1253628,Ginda,IN,30.6945,78.4932,83.17,32,3,6.82,
245,542464,Krasnoarmeysk,RU,51.0239,45.6969,82.54,57,3,8.16,


In [ ]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {"keyword": "hotel", "radius": 5000, "key": g_key}

        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:                
        hotel_df.loc[index, "Hotel Name"] = "NaN"

hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig